### Dependencies

In [3]:
import os
import re
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
import json
from dotenv import load_dotenv

In [7]:
load_dotenv()

In [12]:
STORAGE_ACCOUNT_NAME  = os.getenv("STORAGE_ACCOUNT_NAME")
STORAGE_CONN_STR      = os.getenv("STORAGE_CONN_STR") 
DI_ENDPOINT           = os.getenv("DI_ENDPOINT")
DI_KEY                = os.getenv("DI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 
AZURE_OPENAI_KEY      = os.getenv("AZURE_OPENAI_KEY") 
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")  
AZURE_SEARCH_KEY      = os.getenv("AZURE_SEARCH_KEY")   

### Text Analysis Function

In [ ]:
def analyze_read(endpoint, key, name):
    """Extracts text from a PDF using Azure Document Intelligence."""
    formUrl = f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/bronze/{name}"
    try:
        document_intelligence_client = DocumentIntelligenceClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
        )
        result = poller.result()
        print(f"File {name} analyzed successfully.")
        return result.content
    except Exception as e:
        print(f"Error analyzing file {name}: {e}")

### Text Normalization Function

In [ ]:
def normalize_text(original_text):
    """
    Normalizes the original text for hierarchical chunking,
    without altering the header structure or cutting links.
    """
    lines = original_text.splitlines()
    normalized_lines = []

    # Regular expressions to detect headers
    re_letter_header = re.compile(r'^[A-Z]\.\s+.*')
    re_number_header = re.compile(r'^\d+\.\s+.*')

    i = 0
    while i < len(lines):
        line = lines[i].rstrip("\r\n")  # remove CRLF and trailing whitespace
        # Basic cleaning of space sequences
        line = re.sub(r'\s{2,}', ' ', line)

        # If the line is empty, skip (optional)
        if not line.strip():
            i += 1
            continue

        # 1. Detect broken links (URL split across lines) [optional if needed]
        #    If a line contains 'https://' and the next line is not a header,
        #    you might want to concatenate it.
        #    (Practical example. Adjust logic according to your actual case.)
        if "https://" in line and not re_letter_header.match(line) and not re_number_header.match(line):
            # Check if the URL is cut off. For example, if it doesn't start with https but
            # is "🔗 https://maps.app.goo.gl/" on this line and "RestOfURL" on the next
            # Typically, if the next line is not a header, we could join them.
            # The heuristic here is flexible and depends on your dataset.
            pass  # To keep it simple, we won't do anything here. This is just an example.

        # 2. Add the normalized line
        normalized_lines.append(line.strip())
        i += 1

    # Rebuild text
    cleaned_text = "\n".join(normalized_lines)
    return cleaned_text

### Segmentation Function

In [ ]:
def segment_text_as_json(text):
    """
    Creates a hierarchical structure:
      {
        "A. ...": {
          "1. ...": [...],
          "2. ...": [...]
        },
        "B. ...": {
          "1. ...": [...],
          ...
        },
        ...
      }
    """
    def create_hierarchical_chunking(text):
        # Pattern to detect headers of type 'X. Something...' where X can be A, B, C... or 1, 2, 3...
        # 1) Uppercase letter followed by a period: ^[A-Z]\.\s+(.*)
        # 2) Number (one or more digits) followed by a period: ^\d+\.\s+(.*)
        letter_pattern = re.compile(r'^([A-Z]\.\s+.*)')
        number_pattern = re.compile(r'^(\d+\.\s+.*)')
    
        chunk_dict = {}
        current_letter = None
        current_number = None
    
        # Process line by line
        for line in text.splitlines():
            line = line.strip()
            if not line:
                # If the line is blank, ignore it (it doesn't add content)
                continue
    
            # Matches Letter header (e.g., "A. Text", "B. Text", etc.)
            letter_match = letter_pattern.match(line)
            # Matches Number header (e.g., "1. Text", "2. Text", etc.)
            number_match = number_pattern.match(line)
    
            if letter_match:
                # New major block (e.g., "A. Names of the couple.")
                current_letter = letter_match.group(1)
                # Create an empty dictionary for that letter
                chunk_dict[current_letter] = {}
                # Reset the sub-block
                current_number = None
    
            elif number_match:
                # Sub-block within the letter (e.g., "1. Bride: Laura...")
                if current_letter is not None:
                    current_number = number_match.group(1)
                    # Create a list to store lines belonging to this sub-block
                    chunk_dict[current_letter][current_number] = []
                else:
                    # If no letter is defined, ignore or handle according to your logic
                    pass
            else:
                # Not a letter or number header: add as content line
                # to the last detected sub-block (letter->number)
                if current_letter is not None and current_number is not None:
                    chunk_dict[current_letter][current_number].append(line)
                else:
                    # If we get here without 'current_letter' or 'current_number',
                    # it means the line is not under any valid header.
                    # It can be ignored or handled differently.
                    pass
    
        return chunk_dict
    
    # 2) Create the chunking structure
    hierarchical_chunking = create_hierarchical_chunking(text)
        
    # 3) Convert to JSON to observe the final result
    # and print it (or save it, depending on your use case)
        
    json_result = json.dumps(
        hierarchical_chunking,
        ensure_ascii=False,
        indent=2)
    
    return json_result

### Function to Download Files from a Blob

In [ ]:
def download_from_blob(connection_string, container_name, blob_name):
    """Downloads data from Blob Storage via Connection String"""
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_text = blob_client.download_blob().content_as_text()
        print("File downloaded successfully")
        return blob_text
    except Exception as e:
        print(f"Error downloading file: {e}")

### Function to Upload Files to a Blob

In [ ]:
def upload_to_blob(connection_string, container_name, blob_name, data):
    """Uploads data to Azure Blob Storage using Connection String."""
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_client.upload_blob(data, overwrite=True)
        print(f"File uploaded successfully to {blob_name}")
    except Exception as e:
        print(f"Error uploading file: {e}")

### Text json to python dictionary Function

In [155]:
def string_to_json(json_string):
    try:
        json_data = json.loads(json_string)  # Convertir string a JSON
        return json_data
        
    except json.JSONDecodeError as e:
        print(f"Error al convertir string a JSON: {e}")
        return None

### PDF to text

In [ ]:
# Credentials
endpoint = DI_ENDPOINT
key = DI_KEY

# Values
name = "instrucciones.pdf"
connection_string = STORAGE_CONN_STR      
output_container = "silver"
file_name = os.path.splitext(name)[0]
file_name = f"{file_name}.txt"

# Save PDF file and save the result into the blob
pdf_text = analyze_read(endpoint, key, name)
upload_to_blob(connection_string, output_container, file_name, pdf_text)
print(f"El archivo {file_name} ha sido cargado exitosamente.")

Archivo instrucciones.pdf analizado correctamente.
Archivo subido exitosamente a instrucciones.txt
El archivo instrucciones.txt ha sido cargado exitosamente.


In [158]:
print(pdf_text)

A. Nombres de los novios.
1. Novia: Laura Guadalupe Zarazúa Arvizu.
2. Novio: José Alberto Lozano Sánchez.
B. Cuál es la temática de la bosa?
1. Temática de la boda
- Viajes y ciudades del mundo
C. Actividades de la boda / fiesta.
1. Lista de actividades
- Banquete
- Proyección de video de los novios
- Música en vivo
- Rifa de centros de mesa
- Lanzamiento de ramo
- DJ
- Etc.
D. Código de vestimenta (Etiqueta).
1. Vestimenta y recomendaciones adicionales
- Hombres: Traje o guayabera elegante.
- Mujeres: Vestido formal o de cóctel.
- Evitar colores blancos o beige.
- Procura llevar *zapatos cerrados o de piso* para la recepción.
- Lleva un *suéter o ropa abrigadora* para la noche.
- * No molestar ni aventar comida a los perritos* que estarán en su corral.
- Si necesitas maquillarte, consulta con tiempo a la novia para hacer una *reservación en salones de belleza locales *.
- ¡ Si te animas, puedes traer tu *casa de campaña *! Hay espacio en el terreno para acampar y continuar la fiesta.

### Normalize text

In [ ]:
try:
    connection_string = STORAGE_CONN_STR
    container_name = "silver"
    input_blob_name = "instrucciones.txt"
    output_blob_name = "instrucciones_cleaned.txt"

    # Get original content from the Blob
    blob_text = download_from_blob(connection_string, container_name, input_blob_name)
    # Clean text
    cleaned_text = normalize_text(blob_text)
    # Save the cleaned text into a new Blob Storage file
    upload_to_blob(connection_string, container_name, output_blob_name, cleaned_text)

    print(f"El archivo {output_blob_name} ha sido generado exitosamente en silver.")

except Exception as e:
    print(f"Error al limpiar y guardar el archivo {output_blob_name}: {e}")       

Archivo descargado exitosamente
Archivo subido exitosamente a instrucciones_cleaned.txt
El archivo instrucciones_cleaned.txt ha sido generado exitosamente en silver.


In [161]:
print(cleaned_text)

A. Nombres de los novios.
1. Novia: Laura Guadalupe Zarazúa Arvizu.
2. Novio: José Alberto Lozano Sánchez.
B. Cuál es la temática de la bosa?
1. Temática de la boda
- Viajes y ciudades del mundo
C. Actividades de la boda / fiesta.
1. Lista de actividades
- Banquete
- Proyección de video de los novios
- Música en vivo
- Rifa de centros de mesa
- Lanzamiento de ramo
- DJ
- Etc.
D. Código de vestimenta (Etiqueta).
1. Vestimenta y recomendaciones adicionales
- Hombres: Traje o guayabera elegante.
- Mujeres: Vestido formal o de cóctel.
- Evitar colores blancos o beige.
- Procura llevar *zapatos cerrados o de piso* para la recepción.
- Lleva un *suéter o ropa abrigadora* para la noche.
- * No molestar ni aventar comida a los perritos* que estarán en su corral.
- Si necesitas maquillarte, consulta con tiempo a la novia para hacer una *reservación en salones de belleza locales *.
- ¡ Si te animas, puedes traer tu *casa de campaña *! Hay espacio en el terreno para acampar y continuar la fiesta.

### Chunking

In [ ]:
try:
    connection_string = STORAGE_CONN_STR
    input_container_name = "silver"
    output_container_name = "gold"
    input_blob_name = "instrucciones_cleaned.txt" 
    output_blob_name = "instrucciones_segmented.json"
    
    # Get original content from the Blob
    blob_text = download_from_blob(connection_string, input_container_name, input_blob_name)
    # Chunk text
    chunks = segment_text_as_json(blob_text)
    # Save the cleaned text into a new Blob Storage file
    upload_to_blob(connection_string, output_container_name, output_blob_name, chunks)
    print(f"El archivo {output_blob_name} ha sido generado exitosamente en gold.")

except Exception as e:
    print(f"Error en la segmentación del archivo: {e}")

Archivo descargado exitosamente
Archivo subido exitosamente a instrucciones_segmented.json
El archivo instrucciones_segmented.json ha sido generado exitosamente en gold.


In [164]:
print(chunks)

{
  "A. Nombres de los novios.": {
    "1. Novia: Laura Guadalupe Zarazúa Arvizu.": [],
    "2. Novio: José Alberto Lozano Sánchez.": []
  },
  "B. Cuál es la temática de la bosa?": {
    "1. Temática de la boda": [
      "- Viajes y ciudades del mundo"
    ]
  },
  "C. Actividades de la boda / fiesta.": {
    "1. Lista de actividades": [
      "- Banquete",
      "- Proyección de video de los novios",
      "- Música en vivo",
      "- Rifa de centros de mesa",
      "- Lanzamiento de ramo",
      "- DJ",
      "- Etc."
    ]
  },
  "D. Código de vestimenta (Etiqueta).": {
    "1. Vestimenta y recomendaciones adicionales": [
      "- Hombres: Traje o guayabera elegante.",
      "- Mujeres: Vestido formal o de cóctel.",
      "- Evitar colores blancos o beige.",
      "- Procura llevar *zapatos cerrados o de piso* para la recepción.",
      "- Lleva un *suéter o ropa abrigadora* para la noche.",
      "- * No molestar ni aventar comida a los perritos* que estarán en su corral.",
      

### Upload Chunks json into AI Search index 

In [166]:
import json
import requests
import openai
import os
import uuid
from openai import AzureOpenAI

In [ ]:
embedding_model = "text-embedding-ada-002"

#Creating an Azure OpenAI client
client = AzureOpenAI(
  api_key = AZURE_OPENAI_KEY,  
  api_version = "2024-02-15-preview",
  azure_endpoint = AZURE_OPENAI_ENDPOINT 
)

In [ ]:
# Azure Search Configuration
search_service_endpoint = AZURE_SEARCH_ENDPOINT
index_name = "wedding-info-index"
search_api_key = AZURE_SEARCH_KEY 
api_search_version = "2024-11-01-preview"

headers = {
    "Content-Type": "application/json",
    "api-key": search_api_key
}

In [169]:
# Embedding function
def generate_embedding(client, text, embedding_model):
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']

In [ ]:
# Dowload json from GOLD
connection_string = "DefaultEndpointsProtocol=https;AccountName=fs19920811dev;AccountKey=+EA7jcWlksfpZnod9P+5+9yRlhMf2EFa7ulsO04+Je7w3UyXD/pTL2jCnOGjjJ4THXIp6qEvmDP4+AStVVtjSg==;EndpointSuffix=core.windows.net"
container_name = "gold"
chunks_blob = "instrucciones_segmented.json" 
index_def_blob = "index_definition.json" 

try:
    chunks = download_from_blob(connection_string, container_name, chunks_blob)
    print(f"El archivo {chunks_blob} ha sido leido exitosamente desde gold.")

except Exception as e:
    print(f"Error al leer archivo {chunks_blob}: {e}")

try:
    index_definition = download_from_blob(connection_string, container_name, index_def_blob)
    print(f"El archivo {index_def_blob} ha sido leido exitosamente desde gold.")

except Exception as e:
    print(f"Error al leer archivo {index_def_blob}: {e}")

Archivo descargado exitosamente
El archivo instrucciones_segmented.json ha sido leido exitosamente desde gold.
Archivo descargado exitosamente
El archivo index_definition.json ha sido leido exitosamente desde gold.


In [171]:
def get_index_document(client, chunks, embedding_model):
            
    data = string_to_json(chunks)
    documents = []
    for section_title, subsections in data.items():
        for subsection_title, content_list in subsections.items():
            content_text = " ".join(content_list) if isinstance(content_list, list) else content_list
            text_4_vector = section_title + " " + subsection_title 
            vector = generate_embedding(client, text_4_vector, embedding_model)

            doc = {
                "id": str(uuid.uuid4()),
                "title": section_title,
                "subtitle": subsection_title,
                "content": content_text,
                "contentVector": vector,
                "category": "Instrucciones" if "Instrucciones" in section_title else "General",
                "additionalMetadata": "origen=instrucciones_segmented.json"
            }
            documents.append(doc)
            
    return documents

In [172]:
documents = get_index_document(client, chunks, embedding_model)

In [173]:
len(documents)

48

### Index Definition body

```json
{
  "@odata.etag": "\"0x8DD5E1545012FF4\"",
  "name": "wedding-info-index",
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "searchable": false,
      "filterable": false,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": true,
      "synonymMaps": []
    },
    {
      "name": "title",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "subtitle",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "content",
      "type": "Edm.String",
      "searchable": true,
      "filterable": false,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "contentVector",
      "type": "Collection(Edm.Single)",
      "searchable": true,
      "filterable": false,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "dimensions": 1536,
      "vectorSearchProfile": "vector-profile-19920811",
      "synonymMaps": []
    },
    {
      "name": "category",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": true,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "additionalMetadata",
      "type": "Edm.String",
      "searchable": false,
      "filterable": true,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "synonymMaps": []
    }
  ],
  "scoringProfiles": [],
  "suggesters": [],
  "analyzers": [],
  "normalizers": [],
  "tokenizers": [],
  "tokenFilters": [],
  "charFilters": [],
  "similarity": {
    "@odata.type": "#Microsoft.Azure.Search.BM25Similarity"
  },
  "vectorSearch": {
    "algorithms": [
      {
        "name": "vector-config-19920811",
        "kind": "hnsw",
        "hnswParameters": {
          "metric": "cosine",
          "m": 4,
          "efConstruction": 400,
          "efSearch": 500
        }
      }
    ],
    "profiles": [
      {
        "name": "vector-profile-19920811",
        "algorithm": "vector-config-19920811"
      }
    ],
    "vectorizers": [],
    "compressions": []
  }
}

### Delete the previous index

In [ ]:
# URL del el índice
url = f"{search_service_endpoint}/indexes/{index_name}?api-version={api_search_version}"

# Enviar la solicitud `DELETE` para eliminar el indice previo
response = requests.delete(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 204:
    print(f"Index '{index_name}' was deleted in Azure AI Search.")
elif response.status_code == 404:
    print(f"Index '{index_name}' was not found. It could have been deleted.")
else:
    print(f"Error during Index deletion: {response.text}")

Índice 'wedding-info-index' eliminado correctamente en Azure AI Search.


### Create a new index definition

In [ ]:
# Index definition
index_def_json = string_to_json(index_definition)
# Send `PUT` request to create the index
response = requests.put(url, headers=headers, json=index_def_json)

# Verify if the request was succesful
if response.status_code == 201:
    print(f"Index '{index_name}' created in Azure AI Search.")
elif response.status_code == 204:
    print(f"Index '{index_name}' already exists and was updated succesfully.")
else:
    print(f"Error during Index creation: {response.text}")

Índice 'wedding-info-index' creado correctamente en Azure AI Search.


### Subir documentos al indice

In [ ]:
# Upload data into the index in batch
batch_size = 1
for i in range(0, len(documents), batch_size):
    batch = {"value": documents[i:i+batch_size]}
    url = f"{search_service_endpoint}/indexes/{index_name}/docs/index?api-version={api_search_version}"
    response = requests.post(url, headers=headers, json=batch)
    if response.status_code == 200:
        print(f"Batch {i}-{i+len(batch['value'])} was uploaded.")
    else:
        print(f"Error in the batch {i}: {response.status_code}")

Batch 0-1 subido correctamente.
Batch 1-2 subido correctamente.
Batch 2-3 subido correctamente.
Batch 3-4 subido correctamente.
Batch 4-5 subido correctamente.
Batch 5-6 subido correctamente.
Batch 6-7 subido correctamente.
Batch 7-8 subido correctamente.
Batch 8-9 subido correctamente.
Batch 9-10 subido correctamente.
Batch 10-11 subido correctamente.
Batch 11-12 subido correctamente.
Batch 12-13 subido correctamente.
Batch 13-14 subido correctamente.
Batch 14-15 subido correctamente.
Batch 15-16 subido correctamente.
Batch 16-17 subido correctamente.
Batch 17-18 subido correctamente.
Batch 18-19 subido correctamente.
Batch 19-20 subido correctamente.
Batch 20-21 subido correctamente.
Batch 21-22 subido correctamente.
Batch 22-23 subido correctamente.
Batch 23-24 subido correctamente.
Batch 24-25 subido correctamente.
Batch 25-26 subido correctamente.
Batch 26-27 subido correctamente.
Batch 27-28 subido correctamente.
Batch 28-29 subido correctamente.
Batch 29-30 subido correctamente.

In [ ]:
def chat_GPT(query):
    
    query_embedding = generate_embedding(client, query, embedding_model)
    # Construir consulta híbrida
    payload = {
      "search": query,
      "select": "title, subtitle, content, category", 
      "queryLanguage": "en-us",
      "vectorQueries": [
        {
          "kind": "vector",
          "vector": query_embedding,  
          "fields": "contentVector",
          "k": 3
        }
      ],
      "top": 10
    }
    
    # Enviar consulta a Azure AI Search
    url = f"{search_service_endpoint}/indexes/{index_name}/docs/search?api-version={api_search_version}"
    response = requests.post(url, headers=headers, json=payload)
    
    results = response.json()['value']
    context = " ".join([doc["category"] + ", " + doc["title"] + ", " + doc["subtitle"]  + ", " + doc["content"] for doc in results])
    
    # Generar respuesta con GPT-4o
    
    system_prompt = f"""Act as a virtual assistant for Laura and Alberto's wedding named Sofia. Address each user's inquiry with a friendly and helpful tone.
                        Use the context variable {context} and only respond to questions related to this context.
                        When using bullet points, include an emoji that matches the context of each point.
                        Suggest relevant wedding topics in your responses to guide the user.
                        Act as a guide, providing step-by-step directions from the user's current location to any venue they ask about."""
    
    user_prompt = query
    gpt_response  = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=5000,  
        temperature=0.7,  
        top_p=0.9,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False  
    )
    
    return gpt_response.choices[0].message.content
    return results


In [ ]:
query = "Hi, what is the wedding's location?"
response = chat_GPT(query)
print(response)

¡Hola! 😊 Estoy aquí para ayudarte con todo lo relacionado al transporte para la boda de Laura y Alberto. Aquí tienes los detalles más importantes:

### 🚍 **Servicio de transporte para la boda**
- **Punto de encuentro y hora de salida**:
  - **Lugar**: Walmart Juriquilla (junto a Plaza Uptown).
  - **Dirección**: P.º de la República 12402, Epigmenio González, 76147 Santiago de Querétaro, Qro.
  - [Ubicación del punto de encuentro](https://maps.app.goo.gl/SM3ZydN2Hi34rh7R8).
  - **Hora de partida**: 12:00 p.m. en punto.
  - **Sugerencia**: Llega a las 11:45 a.m. para abordar con calma.

- **Itinerario del transporte**:
  - 🚩 Salida de Walmart Juriquilla.
  - 🕊️ Traslado a la ceremonia religiosa / misa.
  - 🌳 Traslado a la recepción (jardín / terreno).
  - 🏨 Regreso al hotel al finalizar la fiesta.

- **¡Importante!** Llena el formulario para confirmar tu lugar en el transporte:
  - [Formulario del transporte](https://forms.gle/F7vtf8cbaZtijC4T9).

Si tienes dudas adicionales sobre cómo l